In [4]:
from api import build_data



# ----------------------------------------------
# Get data, concate, merge and extract metrics
# for portfolio Dashboard 
# ----------------------------------------------

[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  2 of 2 completed
 DMA
   €
 ZEN
   $
   BTC
   ESE
   GOLD
   GREENBULL
   USDT
   €

# ----------------------------------------------
# Dashboard construction
# by Streamlit
# ----------------------------------------------



NameError: name 's' is not defined

In [5]:
df = build_data()
df

NameError: name 'build_data' is not defined